In [14]:
!pip install faiss-cpu chromadb google-generativeai

In [2]:
import os
import yfinance as yf
import faiss
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import drive
import google.generativeai as genai

In [14]:
drive.mount('/content/drive')

In [4]:
# Paths for FAISS and ChromaDB storage
FAISS_PATH = "/content/drive/MyDrive/faiss_index.bin" # Path to FAISS
CHROMA_PATH = "/content/drive/MyDrive/chroma_db" # Path to CHROMA DB

In [5]:
# Load or initialize FAISS index
embedding_dim = 384  # Dimension for MiniLM embeddings
if os.path.exists(FAISS_PATH):
    index = faiss.read_index(FAISS_PATH)
else:
    index = faiss.IndexFlatL2(embedding_dim)

In [6]:
# Load or initialize ChromaDB
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
chroma_collection = chroma_client.get_or_create_collection(name="stocks")

In [14]:
# Load sentence transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [8]:
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return info.get("longBusinessSummary", "No summary available.")

In [9]:
def update_embeddings(tickers):
    for ticker in tickers:
        if not chroma_collection.get(ticker):
            summary = get_stock_data(ticker)
            embedding = model.encode(summary).tolist()
            chroma_collection.add(id=ticker, embeddings=[embedding])
            index.add(np.array([embedding]))
    faiss.write_index(index, FAISS_PATH)

In [14]:
# Ask user for investment preferences
preferences = input("Enter your investment preferences (e.g., high ROI, low volatility, sustainable growth): ")
pref_embedding = model.encode(preferences).reshape(1, -1)

In [11]:
# Retrieve the best-matching stocks
_, indices = index.search(pref_embedding, k=5)

In [12]:
# Filter out invalid indices (-1) before retrieving tickers
valid_indices = [idx for idx in indices[0] if idx != -1]

In [13]:
# Default stock recommendations
default_stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

In [14]:
# Default stock recommendations
default_stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

# Check if there are valid recommendations
if not valid_indices:
    print("No relevant stock data found based on your preferences.")
    print("Here are 5 top stock recommendations instead:")
    recommended_tickers = default_stocks
else:
    # Retrieve documents using IDs from valid_indices
    recommended_tickers = [chroma_collection.get(ids=[str(idx)])['ids'][0] for idx in valid_indices]

In [ ]:
# Print recommended stocks
print("Top 5 Recommended Stocks:")
for ticker in recommended_tickers:
    print(f"- {ticker}")

In [ ]:
class GeminiAPI:
    def __init__(self, api_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-1.5-flash")

    def explain_stock_choice(self, ticker, preferences):
        prompt = f"Explain why someone interested in {preferences} might choose to invest in {ticker}."
        response = self.model.generate_content(prompt)
        return response.text

In [ ]:
from IPython.display import Markdown

gemini = GeminiAPI(api_key="") # Use your key

# Initialize an empty string to store the markdown text
markdown_text = "# Recommended Stocks\n\n"

# Fetch explanations and store them in markdown format
for ticker in recommended_tickers:
    explanation = gemini.explain_stock_choice(ticker, preferences)
    markdown_text += f"## Stock: {ticker}\n### Reason:\n{explanation}\n\n"

# Display the markdown output
display(Markdown(markdown_text))